In [1]:
import math
import numpy as np

In [2]:
def eval_rest_inequality_functions(f_rest_inequality,x):
    eval_f_rest_inequality = f_rest_inequality(x)
    print(eval_f_rest_inequality)
    #get values that are nonnegative
    idx_zeros = np.logical_and(eval_f_rest_inequality < np.nextafter(0,1),
                               eval_f_rest_inequality > -np.nextafter(0,1))
    idx_positive = eval_f_rest_inequality > 0 
    idx  = np.logical_or(idx_zeros, idx_positive) 
    print(idx)
    eval_f_rest_inequality = np.log(-eval_f_rest_inequality) #this line produces warning
                                                                 #if a value of restrictions is nonnegative
    eval_f_rest_inequality[idx] = 1e10 #large value for values positive or equal to 0    
    return eval_f_rest_inequality

In [3]:
logarithmic_barrier = lambda x, t_path: t_path*f(x) + np.sum(eval_rest_inequality_functions(f_rest_ineq,x))

In [4]:
def f_rest_ineq(x):
    first_rest = lambda x: 4*x[0] + 5*x[1] -1
    second_rest = lambda x: 3*x[0]**2 -3*x[1] -10
    third_rest = lambda x: -x[1]
    fourth_rest = lambda x: 1e-306
    return np.array([first_rest(x), second_rest(x),
                     third_rest(x),
                     fourth_rest(x)])

In [5]:
x_ex = np.array([1,1])

In [6]:
f_rest_ineq(x_ex) 

array([ 8.e+000, -1.e+001, -1.e+000,  1.e-306])

In [7]:
f = lambda x: 3*x[0]**2 + 5*x[1]**2

In [8]:
logarithmic_barrier(x_ex,1)

[ 8.e+000 -1.e+001 -1.e+000  1.e-306]
[ True False False  True]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in log
  # Remove the CWD from sys.path while we load stuff.


20000000010.302586